In [1]:
import mbuild as mb
from mbuild.formats.hoomd_simulation import create_hoomd_simulation
import hoomd
from utils.cg_utils import gsd_rdf
from utils.draw_scene import snap_render, display_movie
import numpy as np
import matplotlib.pyplot as plt

/home/chris/miniconda3/envs/mosdev/lib/python3.7/site-packages/hoomd/meta.py:26: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [10]:
# Use Hoomd to build a simple LJ-particle system:

def build_system(packing='bcc'):
    '''
    Generate a simple LJ particle system using hoomd's create_lattice function
    packing 
    '''
    #hoomd.context.initialize("")
    if packing == 'fcc':
        system = hoomd.init.create_lattice(unitcell=(hoomd.lattice.fcc(a=1.58)), n=6)
    elif packing == 'bcc':
        system = hoomd.init.create_lattice(unitcell=(hoomd.lattice.bcc(a=1.29)), n=6)
    hoomd.dump.gsd('{}_system.gsd'.format(packing), group = hoomd.group.all(), period=None, overwrite=True)
    return system

def hoomd_simulation(system, temp, tau):
    
    hoomd.context.initialize("")
    system = build_system(packing='bcc')
    nl = hoomd.md.nlist.cell()
    lj = hoomd.md.pair.lj(r_cut=2.5, nlist=nl)
    lj.pair_coeff.set('A', 'A', alpha=1.0, epsilon=1.0, sigma=1.0)
    hoomd.md.integrate.mode_standard(dt=0.001)
    _all = hoomd.group.all()
    nvt = hoomd.md.integrate.nvt(group=_all, kT=temp, tau=tau)
    nvt.randomize_velocities(seed=23)
    hoomd.analyze.log(filename='out.log',
                      quantities=["time", "temperature", "potential_energy"],
                      period=100,
                      overwrite=True
                     )
    hoomd.dump.gsd('tau-trajectory.gsd', period=5e3, group=_all, overwrite=True)
    hoomd.run(3e5)

In [4]:
bcc = build_system()

notice(2): Group "all" created containing 1024 particles


In [16]:
hoomd_simulation(system=bcc, temp=1.0, tau=1.0)

notice(2): Group "all" created containing 1024 particles
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 1024
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:10 | Step 11273 / 300000 | TPS 1127.29 | ETA 00:04:16
Time 00:00:20 | Step 23058 / 300000 | TPS 1178.46 | ETA 00:03:55
Time 00:00:30 | Step 34846 / 300000 | TPS 1178.72 | ETA 00:03:44
Time 00:00:40 | Step 46596 / 300000 | TPS 1174.43 | ETA 00:03:35
Time 00:00:50 | Step 58151 / 300000 | TPS 1155.44 | ETA 00:03:29
Time 00:01:00 | Step 69841 / 300000 | TPS 1169 | ETA 00:03:16
Time 00:01:10 | Step 81501 / 300000 | TPS 1165.21 | ETA 00:03:07
Time 00:01:20 | Step 93398 / 300000 | TPS 1189.64 | ETA 00:02:53
Time 00:01:30 | Step 105286 / 300000 | TPS 1188.72 | ETA 00:02:43
Time 00:01:40 | Step 117067 / 300000 | TPS 1178.04 | ETA 00:02:35
Time 00:01:50 | Step 128896 / 300000 | TPS 1182.87 |